<p>Class:  USC Viterbi Data Analytics Bootcamp</p>
<p>Team:  Analyticus (aka Team 5)</p>
<p>Module:  merge_plot_data.py<p>
<p>Version:  March 31, 2018
<p>Input 1:  CDC Influenza-Like-Illness Json File containing 2017 Flu Season.
<p>Input 2:  HHS Vaccination Json File containing 2017 Flu Season.
<p>Output:  Merged file normalized on HHS week.</p>

In [1]:
# Import dependances.
import json
import pandas as pd
import numpy as np

In [2]:
# Load CDC dataframe from CDC file containing state-level data for the 2017 flu season.
df = pd.read_json('../data/cdc_state.json')

In [3]:
# Inspect the CDC dataframe.
df.head()

,Cases,Percent,State,Week,Year
0,698,0.084642,Alabama,40,2017
1,326,0.039532,Alaska,40,2017
10,23,0.002789,Hawaii,40,2017
100,710,0.086097,Alabama,42,2017
1000,2172,0.263384,Alabama,8,2018


In [6]:
# Sort by columns year, week, and state for processing.
df = df.sort_values(by=['Year', 'Week', 'State'])

In [7]:
# Inspect the CDC dataframe.
df.head()

,Cases,Percent,State,Week,Year
0,698,0.084642,Alabama,40,2017
1,326,0.039532,Alaska,40,2017
2,406,0.049233,Arizona,40,2017
3,37,0.004487,Arkansas,40,2017
4,842,0.102104,California,40,2017


In [8]:
# Normalize CDC week to HHS week.  
# Week 1 for HHS starts nine weeks prior to first week of CDC.
# First week of CDC is calendar week 40, which is coverted to HHS week by adjusting the week.
cases_dict = {}
for i in list(df.index):
    case_dict = {}
    case_dict["flu_cases"] = df.loc[i,'Cases']
    case_dict["flu_percent"] = df.loc[i, 'Percent']
    case_dict["state"] = df.loc[i,'State']
    
    # Normalize CDC Week to Plot Week.
    if df.loc[i, 'Year'] == 2017:
        case_dict["week"] = df.loc[i,'Week'] - 30
    else:
        case_dict["week"] = df.loc[i,'Week'] + 22
       
    cases_dict[str(i)] = case_dict

In [9]:
# Load CDC dataframe from dictionary.
df_cdc = pd.DataFrame(cases_dict)

In [10]:
# Inspect the CDC dataframe.
df_cdc.head()

,0,1,10,100,1000,1001,1002,1003,1004,1005,...,990,991,992,993,994,995,996,997,998,999
flu_cases,698,326,23,710,2172,344,879,332,1348,672,...,548,529,3535,1379,410,11195,177,731,295,302
flu_percent,0.084642,0.0395319,0.0027891,0.0860971,0.263384,0.0417147,0.106591,0.0402595,0.163463,0.0814891,...,0.0664524,0.0641484,0.428667,0.167222,0.0497181,1.35755,0.0214637,0.0886437,0.0357728,0.0366216
state,Alabama,Alaska,Hawaii,Alabama,Alabama,Alaska,Arizona,Arkansas,California,Colorado,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
week,10,10,10,12,30,30,30,30,30,30,...,29,29,29,29,29,29,29,29,29,29


In [11]:
# Swap the dataframe axis so to put CDC attributes into columns.
df_cdc = df_cdc.T

In [12]:
# Inspect the CDC dataframe.
df_cdc.head()

,flu_cases,flu_percent,state,week
0,698,0.084642,Alabama,10
1,326,0.0395319,Alaska,10
10,23,0.0027891,Hawaii,10
100,710,0.0860971,Alabama,12
1000,2172,0.263384,Alabama,30


In [13]:
# Load HHS dataframe from HHS file containing state-level data for the 2017 flu season.
df_hhs = pd.read_json('../data/hhs_state.json')

In [14]:
# Inspect HHS dataframe.
df_hhs.head()

,count,disparity,ethnicity,fips,medicare_status,name,percentage,short_name,week,week_start,year
0,75898,0,T,2,A,Alaska,0.000646,AK,1,05AUG,2017
1,75898,0,T,2,A,Alaska,0.002727,AK,2,12AUG,2017
10,75898,0,T,2,A,Alaska,0.238149,AK,11,14OCT,2017
100,2666921,0,T,6,A,California,0.002571,CA,1,05AUG,2017
1000,113366,0,T,46,A,South Dakota,0.000132,SD,1,05AUG,2017


In [15]:
# Reduce columns to those that are needed.
df_hhs = df_hhs.filter(['count','name','percentage', 'week'], axis=1)

In [16]:
# Inspect HHS dataframe.
df_hhs.head()

,count,name,percentage,week
0,75898,Alaska,0.000646,1
1,75898,Alaska,0.002727,2
10,75898,Alaska,0.238149,11
100,2666921,California,0.002571,1
1000,113366,South Dakota,0.000132,1


In [17]:
# Rename HHS dataframe columns to match CCD dataframe columns.
df_hhs = df_hhs.rename(columns={'count':'vaccinations', 'name':'state', 'percentage':'vac_percent'})

In [18]:
# Inspect HHS dataframe.
df_hhs.head()

,vaccinations,state,vac_percent,week
0,75898,Alaska,0.000646,1
1,75898,Alaska,0.002727,2
10,75898,Alaska,0.238149,11
100,2666921,California,0.002571,1
1000,113366,South Dakota,0.000132,1


In [19]:
# Merge CDC and HHS dataframes into a result dataframe.
result = pd.merge(df_cdc, df_hhs, how='outer', on=['state', 'week'])

In [20]:
# Sort the result dataframe by week and state to aid analysis..
result = result.sort_values(by=['week', 'state'])

In [21]:
# Replace null values with zeroes.
# Null values result when there is no intersection between CDC and HHS data.
result.fillna(0, inplace=True)

In [22]:
# Inspect result dataframe.
result.head()

,flu_cases,flu_percent,state,week,vaccinations,vac_percent
1347,0,0.0,Alabama,1.0,543514.0,0.001551
1200,0,0.0,Alaska,1.0,75898.0,0.000646
1550,0,0.0,Arizona,1.0,609729.0,0.001086
1449,0,0.0,Arkansas,1.0,418858.0,0.000186
1202,0,0.0,California,1.0,2666921.0,0.002571


In [23]:
# Inspect rsult dataframe.
result.tail()

,flu_cases,flu_percent,state,week,vaccinations,vac_percent
219,3419,0.414600,Virginia,33.0,0.0,0.0
220,113,0.013703,Washington,33.0,0.0,0.0
221,50,0.006063,West Virginia,33.0,0.0,0.0
222,58,0.007033,Wisconsin,33.0,0.0,0.0
223,131,0.015886,Wyoming,33.0,0.0,0.0


In [24]:
# Write the result dataframe to a json file to be used by plot modules.
result.to_json('../data/plot_state.json')

In [25]:
# Start the validation process by pulling the plot file into a dataframe.
df_plot = pd.read_json('../data/plot_state.json')

In [29]:
# Sort plot datafram to aid in analysis.
df_plot = df_plot.sort_values(by=['state', 'week'])

In [30]:
# Validate plot dataframe.
df_plot.head()

,flu_cases,flu_percent,state,vac_percent,vaccinations,week
1347,0,0.0,Alabama,0.001551,543514,1
1357,0,0.0,Alabama,0.004508,543514,2
1358,0,0.0,Alabama,0.008945,543514,3
1364,0,0.0,Alabama,0.016826,543514,4
1369,0,0.0,Alabama,0.030227,543514,5


In [31]:
# Validate plot dataframe.
df_plot.tail()

,flu_cases,flu_percent,state,vac_percent,vaccinations,week
1199,302,0.036622,Wyoming,0.0,0,29
57,246,0.029831,Wyoming,0.0,0,30
112,194,0.023525,Wyoming,0.0,0,31
168,194,0.023525,Wyoming,0.0,0,32
223,131,0.015886,Wyoming,0.0,0,33
